In [1]:
from pyspark.sql import SparkSession


In [2]:
#TODO: make a method out of this.

import os
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/jdk1.8.0_241.jdk/Contents/Home'

import subprocess
subprocess.check_output(['java', '-version'], stderr=subprocess.STDOUT)

b'java version "1.8.0_241"\nJava(TM) SE Runtime Environment (build 1.8.0_241-b07)\nJava HotSpot(TM) 64-Bit Server VM (build 25.241-b07, mixed mode)\n'

In [3]:
spark = SparkSession.builder.appName("Basics").getOrCreate()
df = spark.read.parquet("data/final_join.parquet")


AnalysisException: 'Path does not exist: file:/Users/Jonas/PycharmProjects/unif_projectjes/DE_opdracht/opdracht_forum/data/final_join.parquet;'

In [ ]:
df.show()

First we'll drop some columns and impute some others and after that we'll do some basic data analysis, 
largely based on this post: https://medium.com/@aieeshashafique/exploratory-data-analysis-using-pyspark-dataframe-in-python-bd55c02a2852
(part1, this file)

Then We'll do some more advanced EDA, 
after which we'll choose a cutoff value which will allow us to define "an inactive user" (part 2)

In [ ]:
#dropping first- and lastname columns
df = df.select([c for c in df.columns if c not in {'lastName','firstName'}])

# Adding a lot of datetime columns
from pyspark.sql.functions import dayofmonth,dayofweek,dayofyear,year
columns = [dayofmonth,dayofweek,dayofyear,year]
for x in columns:
    df = df.withColumn(f"creationDate_comment_{x.__name__}", x(df['creationDate_comment']))

# extracting domain name from emails
from pyspark.sql.functions import split
df = df.withColumn('email', split(df['email'], '@')[1])

#Hardcoding the year in is not ideal, but for our purposes allowable 
df = df.withColumn('age', 2020 - year(df['birthday'])).drop('birthday')


In [ ]:
df.show()

In [ ]:
# 1.schema of pyspark dataframe

df.printSchema()


In [ ]:
# 2.Show your PySpark Dataframe 

df.show()

In [ ]:
# 3. Count function of PySpark Dataframe

df.count()


In [ ]:
# 4. Statistical Properties of each column (not that interesting for this dataframe, misschien beter zelfs gewoon schrappen)

columns = ['gender', 'browserUsed', 'creationDate_comment_month']
for col in columns:
    print(col)
    df.describe([col]).show()


# 5. Find unique values of a categorical column
columns = ['browserUsed', 'gender']
for x in columns:
    print(df.select(x).distinct().rdd.map(lambda r: r[0]).collect())



In [ ]:
#7. Count the missing values in a column of PySpark Dataframe 


for col in df.columns:
    print(col, "with null values: ", df.filter(df[col].isNull()).count())
    

In [ ]:
df.write.save("data/Data_analysis_part1.parquet", mode='overwrite')


